In [1]:
import os
import time
import math

import pandas as pd
import tenseal as ts

In [2]:
def cleanup():
    for f in [
        "enc_bfv.dat",
        "res_bfv_stats.dat"
    ]:
        if os.path.exists(f):
            os.remove(f)

In [3]:
def setup_bfv():
    context = ts.context(
        ts.SCHEME_TYPE.BFV, 
        poly_modulus_degree=16384,
        plain_modulus=1099511922689, # no overflow
    )
    context.generate_galois_keys()
    context.generate_relin_keys()
    return context

def holder_encrypt_bfv(context, salary, bonus, filename):
    enc_s = ts.bfv_vector(context, salary)
    enc_b = ts.bfv_vector(context, bonus)

    ser_s = enc_s.serialize()
    ser_b = enc_b.serialize()
    
    with open(filename, "wb") as f:
        f.write(len(ser_s).to_bytes(4, 'big'))
        f.write(ser_s)
        f.write(ser_b)

def analyzer_process_bfv(context, in_file, out_file):
    with open(in_file, "rb") as f:
        size_s = int.from_bytes(f.read(4), 'big')
        bytes_s = f.read(size_s)
        bytes_b = f.read()
    
    enc_s = ts.bfv_vector_from(context, bytes_s)
    enc_b = ts.bfv_vector_from(context, bytes_b)
    
    # Calculate sum for salaries and bonuses
    enc_salary_sum = enc_s.sum()
    enc_bonus_sum = enc_b.sum()
    
    # Calculate Result = (salary + 0.1 * bonus) * 1.05
    # Using integer arithmetic: (10*S + B) * 21 / 200
    enc_result = (enc_s.mul(10) + enc_b).mul(21).sum()
    
    # Serialize results
    with open(out_file, "wb") as f:
        ser_sal_sum = enc_salary_sum.serialize()
        ser_bon_sum = enc_bonus_sum.serialize()
        ser_total = enc_result.serialize()
        
        f.write(len(ser_sal_sum).to_bytes(4, 'big'))
        f.write(ser_sal_sum)
        f.write(len(ser_bon_sum).to_bytes(4, 'big'))
        f.write(ser_bon_sum)
        f.write(len(ser_total).to_bytes(4, 'big'))
        f.write(ser_total)

def holder_decrypt_bfv(context, filename):
    with open(filename, "rb") as f:
        # Read salary sum
        size1 = int.from_bytes(f.read(4), 'big')
        enc_sal_sum = ts.bfv_vector_from(context, f.read(size1))
        
        # Read bonus sum
        size2 = int.from_bytes(f.read(4), 'big')
        enc_bon_sum = ts.bfv_vector_from(context, f.read(size2))
        
        # Read total result
        size3 = int.from_bytes(f.read(4), 'big')
        enc_total = ts.bfv_vector_from(context, f.read(size3))
    
    salary_sum = enc_sal_sum.decrypt()[0]
    bonus_sum = enc_bon_sum.decrypt()[0]
    total_numerator = enc_total.decrypt()[0]
    
    # Apply the division by 200 (from the 10 * 20 scaling factor)
    total_result = total_numerator / 200.0
    
    return {
        'salary_sum': salary_sum,
        'bonus_sum': bonus_sum,
        'total_result': total_result
    }

In [4]:
# Read dataset
df = pd.read_csv("datasets/dataset.csv")
salaries_list = df["salary_cents"].tolist()
bonus_list = df["bonus_cents"].tolist()

print(f"Dataset Size: {len(salaries_list)} rows")

# Ground Truth
salary_sum_gt = sum(salaries_list)
bonus_sum_gt = sum(bonus_list)
total_result_gt = sum([(s + 0.1 * b) * 1.05 for s, b in zip(salaries_list, bonus_list)])

Dataset Size: 8192 rows


In [6]:
print("=" * 80)
print("BFV - STATISTICAL ANALYSIS")
print("=" * 80)

# Cleanup any existing files before execution
cleanup()

# Execution BFV Statistics
ctx_bfv = setup_bfv()

start = time.time()
holder_encrypt_bfv(ctx_bfv, salaries_list, bonus_list, "enc_bfv.dat")
t_enc_bfv = time.time() - start

start_proc = time.time()
analyzer_process_bfv(ctx_bfv, "enc_bfv.dat", "res_bfv_stats.dat")
t_proc_bfv = time.time() - start_proc

start_dec = time.time()
bfv_stats = holder_decrypt_bfv(ctx_bfv, "res_bfv_stats.dat")
t_dec_bfv = time.time() - start_dec

print(f"\n--- Timing ---")
print(f"Encryption:  {t_enc_bfv:.4f}s")
print(f"Processing:  {t_proc_bfv:.4f}s")
print(f"Decryption:  {t_dec_bfv:.4f}s")
print(f"Total Time:  {t_enc_bfv + t_proc_bfv + t_dec_bfv:.4f}s")

# ==============================================================================
# VERIFICATION
# ==============================================================================

print("\n--- Statistics Verification ---")
print(f"\nSalary Sum:")
print(f"  Ground Truth: {salary_sum_gt}")
print(f"  BFV Result:   {bfv_stats['salary_sum']}")
print(f"  Error:        {abs(salary_sum_gt - bfv_stats['salary_sum'])}")

print(f"\nBonus Sum:")
print(f"  Ground Truth: {bonus_sum_gt}")
print(f"  BFV Result:   {bfv_stats['bonus_sum']}")
print(f"  Error:        {abs(bonus_sum_gt - bfv_stats['bonus_sum'])}")

print(f"\nTotal Result:")
print(f"  Ground Truth: {total_result_gt:.10f}")
print(f"  BFV Result:   {bfv_stats['total_result']:.10f}")
print(f"  Error:        {abs(total_result_gt - bfv_stats['total_result']):.10f}")

BFV - STATISTICAL ANALYSIS

--- Timing ---
Encryption:  0.0462s
Processing:  0.8684s
Decryption:  0.0570s
Total Time:  0.9716s

--- Statistics Verification ---

Salary Sum:
  Ground Truth: 2244383855
  BFV Result:   2244383855
  Error:        0

Bonus Sum:
  Ground Truth: 406517633
  BFV Result:   406517633
  Error:        0

Total Result:
  Ground Truth: 2399287399.2150001526
  BFV Result:   2399287399.2150001526
  Error:        0.0000000000
